In [1]:
import librosa
import numpy as np
import wave
import soundfile as sf
import os


def split_audio_file(audio_name, audio_file, subfile_duration, num_decimal_places=0):
    """
    将1个音频文件切割成任意长度的WAV文件并输出。

    参数:
    audio_file (str): 音频文件的路径。
    subfile_duration (float): 每个子文件的长度，以秒为单位。
    num_decimal_places (int, optional): 保留小数点后几位。默认为0。

    返回:
    None
    """
    # 加载音频文件
    # mono = Flase会保留双通道
    y, sr = librosa.load(audio_file, sr=None, mono=False)
    y = amplitude(y)
    duration = librosa.get_duration(y=y, sr=sr)

    # 确定子文件的数量
    num_subfiles = int(duration // subfile_duration)

    # 利用NumPy的linspace函数创建一系列时间戳，用于切割音频
    timestamps = np.linspace(0, num_subfiles * subfile_duration, num=num_subfiles + 1, endpoint=True)

    print(f'{audio_name} split start !')

    # 循环遍历每个子文件，并使用librosa.output.write_wav函数将它们保存为.wav文件
    for i in range(num_subfiles):
        start = int(timestamps[i] * sr)
        end = int(timestamps[i + 1] * sr)
        subfile = y[:, start:end]
        filename = f'{audio_name}_{i + 1:0{len(str(num_subfiles))}d}.wav'
        write_path = './output/split_audio/' + filename
        sf.write(write_path, np.transpose(subfile), sr, format='wav', subtype='PCM_16')

    print(f'{num_subfiles} subfiles have been created.')

    # 如果剩余部分长度不足一个子文件，则将其丢弃
    remaining_duration = duration - num_subfiles * subfile_duration
    if remaining_duration >= subfile_duration:
        num_subfiles += 1
        start = int(timestamps[-1] * sr)
        end = len(y)
        subfile = y[:, start:end]
        filename = f'{audio_name}_{i + 1:0{len(str(num_subfiles))}d}.wav'
        write_path = './output/split_audio/' + filename
        sf.write(write_path, np.transpose(subfile), sr, format='wav', subtype='PCM_16')
        # librosa.output.write_wav(filename, subfile, sr)
        print(f'Remaining duration of {remaining_duration:.{num_decimal_places}f}s has been saved as {filename}.')


In [2]:
def amplitude(y):
    """
    Applies automatic gain control to the audio to make it sound more comfortable to listen to.

    Parameters:
    ----------
    y : numpy.ndarray [shape=(n_samples,)]
        The input audio signal as a time series.
    sr : number > 0 [scalar]
        The sampling rate of `y`, i.e. the number of samples per second.

    Returns:
    -------
    y_agc : numpy.ndarray [shape=(n_samples,)]
        The output audio signal after applying AGC.
    """
    target_db = -40  # 目标分贝值
    db = librosa.amplitude_to_db(y)
    gain = librosa.db_to_amplitude(target_db - db.mean())  # 计算增益系数
    y_agc = y * gain
    return y_agc

In [3]:
# audio_dir = r'./input/新音频'
# audio_dir = r'./input/702_select'
audio_dir = r'./input/新音频_2'
for audio_name in os.listdir(audio_dir):
    audio_path = os.path.join(audio_dir, audio_name)
    audio_name = audio_name.split('.')[0]
    split_audio_file(audio_name, audio_path, 10)

大型车辆 split start !
4 subfiles have been created.
布谷鸟生活声 split start !
30 subfiles have been created.
虫鸣行驶声 split start !
6 subfiles have been created.
鸟鸣小雨 split start !
30 subfiles have been created.
鸟鸣机械声 split start !
6 subfiles have been created.


In [4]:
# audio_path = r'./input/test/sample.wav'
# # y, sr = librosa.load(audio_path, sr=None, mono=False)
# y, sr = librosa.load(audio_path, sr=None, mono=False)
# # sf.write(r'./output/test1.wav', y, sr, format='wav', subtype='PCM_16')
# # sf.write(file=r'./output/test1.wav', data=np.transpose(y), samplerate=sr, format='wav', subtype='PCM_16')
# db1 = librosa.amplitude_to_db(y)

In [5]:
# # 计算增益系数
# target_db = -40  # 目标分贝值
# gain1 = librosa.db_to_amplitude(target_db - db1.mean())
# y1_agc = y * gain1
# # sf.write(r'./output/sample_ad.wav', np.transpose(y1_agc), sr, format='wav', subtype='PCM_16')

In [6]:
# db2 = librosa.amplitude_to_db(y1_agc)
# db2.mean()

In [7]:
import os
import pandas as pd

folder_path = r'./output/split_audio'

files = os.listdir(folder_path)

# 创建一个DataFrame对象，将文件名放在第一列
df = pd.DataFrame({'File Name': files})

# 将DataFrame写入Excel文件
# df.to_excel(r'./output/split_audio_names.xlsx', index=False)
df.to_excel(r'./output/split_audio_names.xlsx', index=False)